In [1]:
from utils.data_preprocess import load_data, split_xy_save, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input

2.12.0
# GPUs Available:  1


In [2]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data = data.drop(columns=["mfc10_residual"], inplace=False)

data['x2'] = data['x2'] / cfg['geometry']['span']
data['x1'] = data['x1'] / cfg['geometry']['span']

data['y2'] = data['y2'] / cfg['geometry']['width']
data['y1'] = data['y1'] / cfg['geometry']['width']

data['x2'] = data['x2'].replace(np.nan, -100)
data['y2'] = data['y2'].replace(np.nan, -100)
# or try generating a random number

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
# y1 = y[['x1', 'y1', 'x2', 'y2']]
# y2 = y[['leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[['leak_1', 'leak_2']]

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

In [4]:
def base_model(inputs):
    x= Dense(128, activation='relu')(inputs)
    x= Dense(128, activation='relu')(x)
    return x

def final_model(inputs):
    x = base_model(inputs)
    y1 = Dense(units='4', name='y1')(x)
    y2 = Dense(units = '2', name = 'y2')(x)
    model = Model(inputs=inputs, outputs = [y1, y2])
    
    return model

inputs = tf.keras.layers.Input(shape=(10,))
model = final_model(inputs)

losses = {
	"y1": "mse",
	"y2": tf.keras.losses.CategoricalCrossentropy()
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }

lossWeights = {"y1": 4.0, "y2": 1.0}

model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
              loss=losses, loss_weights=lossWeights,
                metrics = metrics)
y_train = {
    "y1" : y1_train,
    "y2" : y2_train
}

y_val = {
    "y1" : y1_val,
    "y2" : y2_val
}


history = model.fit(X_train, y_train, validation_data = (X_val, y_val), verbose = 1, epochs=1000)

Epoch 1/1000
12/12 [==============================] - 4s 26ms/step - loss: 2012.1397 - y1_loss: 2009.5293 - y2_loss: 2.6104 - y1_mae: 25.0065 - y2_mae: 1.2976 - val_loss: 487.5357 - val_y1_loss: 486.6624 - val_y2_loss: 0.8732 - val_y1_mae: 14.5818 - val_y2_mae: 4.2073
Epoch 2/1000
12/12 [==============================] - 0s 7ms/step - loss: 239.9969 - y1_loss: 239.2293 - y2_loss: 0.7676 - y1_mae: 8.4928 - y2_mae: 5.7166 - val_loss: 58.5443 - val_y1_loss: 57.8337 - val_y2_loss: 0.7106 - val_y1_mae: 4.1035 - val_y2_mae: 7.5632
Epoch 3/1000
12/12 [==============================] - 0s 8ms/step - loss: 113.7279 - y1_loss: 113.0448 - y2_loss: 0.6831 - y1_mae: 4.6024 - y2_mae: 9.1009 - val_loss: 45.9360 - val_y1_loss: 45.2440 - val_y2_loss: 0.6921 - val_y1_mae: 3.2926 - val_y2_mae: 10.4884
Epoch 4/1000
12/12 [==============================] - 0s 8ms/step - loss: 98.2799 - y1_loss: 97.6013 - y2_loss: 0.6786 - y1_mae: 4.0116 - y2_mae: 10.2056 - val_loss: 45.6418 - val_y1_loss: 44.9453 - val_y2_

In [6]:
y_test = {
    "y1" : y1_test,
    "y2" : y2_test
}
model_evaluate, y_pred = model_eval(model, X_test, y_test, X_train, y_train, X_val, y_val)

3/3 [==============================] - 0s 4ms/step - loss: 129.6387 - y1_loss: 121.1507 - y2_loss: 8.4880 - y1_mae: 1.8529 - y2_mae: 87.8042
